In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [4]:
import re
import ast

In [5]:
import amrlib
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [6]:
from openai import OpenAI
client = OpenAI(api_key='YOUR_KEY')

# Set up YOUR OpenAI API key
#OPENAI_API_KEY = "YOUR_KEY"
# Set MODEL to "gpt-3.5-turbo"
model = "gpt-3.5-turbo"
# Set temperature to 0.9, it can be from 0~1, 0 is the most conservative, 1 is the most creative
temperature = 0.5

In [13]:
messages = [
    {"role": "system", "content": "You are a helpful and honest assistant. \
      Respond concisely and truthfully."},
    {"role": "user", "content": "Explain Einstein's relativity theory in three sentences."},
]

# Send the message to the bot
response_message = client.chat.completions.create(model=model, messages=messages, temperature=temperature)

# Get the response from the bot
response = response_message.choices[0].message.content

print(response)

Einstein's theory of relativity consists of two parts: special relativity and general relativity. Special relativity describes how time and space are intertwined and can be affected by motion. General relativity extends this to include gravity as a curvature of spacetime caused by mass and energy.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from datasets import load_dataset
ds = load_dataset("Idavidrein/gpqa", "gpqa_main")

In [8]:
ds['train'][1]

{'Pre-Revision Question': 'Two quantum states with energies E1 and E2 have a lifetime of 10^-9 sec and 10^-8 sec, respectively. We want to clearly distinguish these two energy levels. Which one of the following options could be their energy difference so that they be clearly resolved?\n',
 'Pre-Revision Correct Answer': '10^-4 ev',
 'Pre-Revision Incorrect Answer 1': '10^-8 ev',
 'Pre-Revision Incorrect Answer 2': '10^-9 ev',
 'Pre-Revision Incorrect Answer 3': '10^-11 ev',
 'Pre-Revision Explanation': 'According to uncertainty principle, Delta E* Delta t=hbar/2. Delta t is the lifetime and Delta E is the width of the energy level . With Delta t=10^-9 s==> Delta E1= 3.3 *10^-7 ev. And Delta t=10^-11 s gives Delta E2=3.3*10^-8 ev.\nTherefore, the energy difference between the two states must be significantly greater than 10^-7 ev. So the answer is 10^-4 ev.',
 'Self-reported question-writing time (minutes)': 30.0,
 'Question': 'Two quantum states with energies E1 and E2 have a lifetime 

In [9]:
def Answer_Agent_IO(question='', num_seq=1):
    message = \
    f"""Given a question, give out the answer from the candidates, without any explanations. \
    Here is an example as guideline:
    "Question: What is the capital of France? \
    Answer: ###Paris###" \
    Make sure to include your answer in the format: ###your answer###. \
    Question: {question} \
    Answer: \
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that answer scientific questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages_in, temperature=temperature, n=num_seq)

    responses = []
    n = 0
    while n < num_seq:
        responses.append(response_message.choices[n].message.content)
        n += 1
    
    return responses

In [10]:
def Answer_Agent_CoT_0Shot(question='', num_seq=1):
    message = \
    f"""Given a question, give out your answer. \
    Make sure to include your final answer in the format: ###your choice###. \
    Question: {question} \
    Let's think step by step. \
    Answer: \
    """
    
    messages_in = [{"role": "system", "content": \
                     "You are a faithful agent that answer scientific questions accurately."},\
                {"role": "user", "content": message}]
    
    response_message = client.chat.completions.create(model=model, messages=messages_in, temperature=temperature, n=num_seq)

    responses = []
    n = 0
    while n < num_seq:
        responses.append(response_message.choices[n].message.content)
        n += 1
    
    return responses

In [22]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [19]:
all_answer_lm3_text_io = []
ori_answers_io = []
all_counts = -1
for s in tqdm(list(ds['train'])[0:100]):
    all_counts += 1
    ori_question = s['Question']

    response = Answer_Agent_IO(ori_question, 1)
    ori_answers_io.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_io.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_io.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_io.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_io.append('')
                continue

 27%|██▋       | 27/100 [00:45<13:10, 10.83s/it]

no answer provided
###AGAGTCAAGAAGGTTTGGGAGGAGAGGAGGGTGGGGGGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAGGAG


100%|██████████| 100/100 [01:19<00:00,  1.26it/s]


In [24]:
all_answer_lm3_text_cot_0s = []
all_counts = -1

for s in tqdm(list(ds['train'])[0:100]):
    all_counts += 1
    ori_question = s['Question']

    response = Answer_Agent_CoT_0Shot(ori_question, 1)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot_0s.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot_0s.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot_0s.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot_0s.append('')
                continue

 23%|██▎       | 23/100 [01:00<02:44,  2.14s/it]

no answer provided
To calculate the inner radius of the torus using the Barvainis model, we can start by calculating the luminosity of the AGN in the UV range. Given the bolometric luminosity of 1.5*10^46 erg/s and a bolometric correction factor of 5 for the UV range, the UV luminosity would be:

UV Luminosity = Bolometric Luminosity / Bolometric Correction Factor
UV Luminosity = 1.5*10^46 erg/s / 5
UV Luminosity = 3*10^45 erg/s

Next, we can use the Barvainis model which relates the inner radius of the torus to the temperature of dust grain sublimation and the luminosity of the AGN. The formula is:

R = sqrt(L / (16 * π * σ * T^4))

Where:
R = Inner radius of the torus
L = Luminosity of the AGN
σ = Stefan-Boltzmann constant (5.67 x 10^-5 erg/(cm^2*s*K^4))
T = Temperature of dust grain sublimation

Plugging in the values:
R = sqrt((3*10^45 erg/s) / (16 * π * (5.67 x 10^-5) * (900 K)^4))

Calculating the inner radius of the torus will give us the final answer.


 61%|██████    | 61/100 [02:26<01:05,  1.68s/it]

no answer provided
For the first reaction:
Reactant A is likely a cyclic compound with the potential to undergo ring expansion or contraction when treated with sulfuric acid (H2SO4).

For the second reaction:
Reactant B reacts with butyllithium (BuLi) and a proton source (H+) to form a tertiary alcohol with a phenyl and a methyl group attached to the carbon chain.

###A: Cyclic compound
B: Alkyl halide###


100%|██████████| 100/100 [04:02<00:00,  2.43s/it]


In [11]:
import sys
import re
import string
from collections import Counter
import pickle

In [12]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [27]:
with open('answers_gpt3_io.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_io))

In [28]:
with open('answers_gpt3_io_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_answers_io))

In [29]:
with open('answers_gpt3_cot_0Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot_0s))

In [32]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['Correct Answer'] for s in list(ds['train'])[0:100]))

In [13]:
with open('answers_golden.txt', 'r') as file:
    golden = file.readlines()
with open('answers_gpt3_io.txt', 'r') as file:
    output_io = file.readlines()
with open('answers_gpt3_cot_0Shot.txt', 'r') as file:
    output_cot = file.readlines()

In [14]:
def evaluation(output, golden):
    correct = 0
    length = len(golden)

    for (o, g) in zip(output, golden):
        if len(o) == 0 or len(g) == 0:
            continue
        o = o[0]
        g = g[0]
        if o == g:
            correct += 1
    
    return correct, correct/length

In [41]:
output_io = [re.sub(r'\n', '', s) for s in output_io]
output_cot = [re.sub(r'\n', '', s) for s in output_cot]
golden = [re.sub(r'\n', '', s) for s in golden]

In [45]:
evaluation(output_io, golden)

(8, 0.06956521739130435)

In [46]:
evaluation(output_cot, golden)

(8, 0.06956521739130435)

In [17]:
def Answer_Agent_debate(ori_question='', rounds=1, num_agents=3):
    message_user_prior = \
    f"""Given a question, give out your chain of thought process and the final answer. \
    Make sure to include your answer in the format: ###your answer###. \
    Give out the answer in numerical format. \
    Question: {ori_question} \
    Let's think step by step. \
    Answer: \
    """

    message = \
    f"""Given some potential answers given by other agents: \
    if you think your previous answer is wrong, and one of the potential answers is correct, \
    provide the correct answer and your reasoning for it; \
    if you think your previous answer is correct, explain why it is correct. \
    Make sure to include your final answer in the format: ###answer###. \
    """
    #    Question: {ori_question} \

    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    
    all_responses = {}
    for i in range(num_agents):
        all_responses[i] = {'user': [], 'assistant': []}
    response_pool = set()
    
    response_list = Answer_Agent_CoT_0Shot(ori_question, num_agents)
    for j in range(len(response_list)):
        all_responses[j]['user'].append(message_user_prior)
        try:
            response_format = re.search('###(.*)###', response_list[j]).group(1)
            all_responses[j]['assistant'].append(response_format)
        except:
            response_format = response_list[j]
            all_responses[j]['assistant'].append('no answer')
    
    for r in range(rounds):
        for i in range(num_agents):
            new_messages = messages
            new_message = message

            new_messages.append({"role": "user", "content": all_responses[i]['user'][0]})
            new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][0]})
            
            if r != 0:
                new_messages.append({"role": "user", "content": all_responses[i]['user'][r]})
                new_messages.append({"role": "assistant", "content": all_responses[i]['assistant'][r]})

            potential_answers = list(response_pool)
            new_message += f'Potential answers: '
            num_ans = 1
            for pa in potential_answers:
                new_message += f'{num_ans}:{pa};'
                num_ans += 1
            new_messages.append({"role": "user", "content": new_message})
            
            response_message = client.chat.completions.create(model=model, messages=new_messages, temperature=temperature)
            response = response_message.choices[0].message.content
            all_responses[i]['user'].append(new_message)
            all_responses[i]['assistant'].append(response)

    #put all answers in a pool, and then do majority vote
    all_responses_pool = []
    for i in range(num_agents):
        answer_i = all_responses[i]['assistant'][-1]
        try:
            response_format = re.search('###(.*)###', answer_i).group(1)
            all_responses_pool.append(response_format)
        except:
            all_responses_pool.append(answer_i)

    return all_responses, max(set(all_responses_pool), key=all_responses_pool.count)

In [18]:
all_answer_gpt3_text_cot_3ensemble = []
ori_answers = []
all_counts = -1
for s in tqdm(list(ds['train'])[0:100]):
    all_counts += 1
    ori_question = s['Question']

    response, final_response = Answer_Agent_debate(ori_question, 2, 3)
    all_answer_gpt3_text_cot_3ensemble.append(final_response)
    ori_answers.append(response)

100%|██████████| 100/100 [18:06<00:00, 10.86s/it]


In [19]:
output_3ens = [re.sub(r'\n', '', s) for s in all_answer_gpt3_text_cot_3ensemble]
golden = [re.sub(r'\n', '', s) for s in golden]

In [20]:
evaluation(output_3ens, golden)

(5, 0.043478260869565216)

In [21]:
output_3ens

['sarcolemma',
 '4.74 x 10^-3 eV',
 '10',
 '4',
 '127.6°',
 '16π',
 'Cannot be determined',
 '8.7',
 "Gauss's law for magnetism and the absence of magnetic monopoles",
 'cyclopentene derivative',
 'ChIP-seq and RNA-seq',
 '28.08 cm3, pH 1.55',
 '$\\begin{pmatrix} \\frac{1}{\\sqrt{2}} \\\\ \\frac{1}{\\sqrt{2}} \\end{pmatrix}$',
 'E_n,l = (n + 1)ħω + (l + 1/2)ħω',
 'Interference',
 '4',
 '1.022 MeV',
 'answer',
 '0.2 mol of carbon',
 "triphenyleno[1,2-c:5,6-c':9,10-c'']trifuran-1,3,6,8,11,13-hexaone",
 'I stand by my previous answer as correct.',
 '(sigma * b) / 2### is the correct answer. The current density induced on the plate due to the variation of the vector potential is given by J = sigma * (∂A/∂t). In this case, the magnetic vector potential A is given as B*r/2 in the phi direction for r < R and 0 for r > R. As B increases linearly with time as B = b*t, the partial derivative of A with respect to time is B = b. Therefore, the magnitude of the current density induced on the plate 